In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.darcy import cc1
from models.darcy import cc2
from models.darcy import bc
import numpy as np
from utils.image_gradient import SobelFilter
from FEA_simp import ComputeTarget

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# random data
input = np.array(np.random.random([1,1,20,40]), dtype=np.float32)
# ComputeTarget是在FEA_simp.py中封装的计算真值的函数
output = torch.from_numpy(np.array(ComputeTarget(input), dtype=np.float32)).to(device)
input = torch.from_numpy(input).to(device)

In [3]:
# real data
data = np.loadtxt("data/rho20x40_SIMP_Edge.txt", dtype=np.float32)
# [bs, 1, 20, 40]
data = data.reshape(-1,1,40,20).transpose([0,1,3,2])

data_u = np.loadtxt("data/dis20x40_SIMP_Edge.txt", dtype=np.float32)
data_s = np.loadtxt("data/stress20x40_SIMP_Edge.txt", dtype=np.float32)

ref_u0 = torch.from_numpy(data_u).unsqueeze(1).to(device)
ref_uy = ref_u0[:,:,range(1,1722,2)]
ref_ux = ref_u0[:,:,range(0,1722,2)]
ref_s0 = torch.from_numpy(data_s).unsqueeze(1).to(device)
ref_sx = ref_s0[:,:,range(0,2583,3)]
ref_sy = ref_s0[:,:,range(1,2583,3)]
ref_sxy = ref_s0[:,:,range(2,2583,3)]
# [bs, 5, 21, 41]
ref = torch.cat([ref_ux, ref_uy, ref_sx, ref_sy, ref_sxy],1).view(-1,5,41,21).permute(0,1,3,2)

In [26]:
# 取部分
input = torch.from_numpy(data[:16])
output = ref[:16]

print(input.shape)
print(output.shape)

torch.Size([16, 1, 20, 40])
torch.Size([16, 5, 21, 41])


In [27]:
# post output
WEIGHTS_2x2 = torch.FloatTensor( np.ones([1,1,2,2])/4 ).to(device)
o0 = F.conv2d(output[:,[0]], WEIGHTS_2x2, stride=1, padding=0, bias=None)
o1 = F.conv2d(output[:,[1]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o2 = F.conv2d(output[:,[2]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o3 = F.conv2d(output[:,[3]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
o4 = F.conv2d(output[:,[4]], WEIGHTS_2x2, stride=1, padding=0, bias=None) 
output_post = torch.cat([o0,o1,o2,o3,o4],1)
print(output_post.shape)

torch.Size([16, 5, 20, 40])


In [28]:
sobel_filter = SobelFilter(64, correct=False, device=device)

In [29]:
print (cc1(input, output, output_post, sobel_filter, device))

tensor(9.7479e-05, device='cuda:0')


In [30]:
print(cc2(output_post, sobel_filter))

tensor(31.0947, device='cuda:0')


In [31]:
loss_boundary = bc(output, output_post)
print(loss_boundary)

tensor(1.2465, device='cuda:0')


## loss detail

In [ ]:
# loss_boundary detail
ux = output[:, [0]]
uy = output[:, [1]]
lu = ux[:,:,:,0]**2 + uy[:,:,:,0]**2
sx = output[:, [2]]
sy = output[:, [3]]
sxy = output[:, [4]]
lbr = (sx[:,:,:,40]-1)**2 + (sxy[:,:,:,40])**2
lbt = (sy[:,:,0,2:-2])**2 + (sxy[:,:,0,2:-2])**2
lbb = (sy[:,:,20,2:-2])**2 + (sxy[:,:,20,2:-2])**2

In [ ]:
print(lu.sum([-1]).mean())
print(lbr.sum([-1]).mean())
print(lbt.sum([-1]).mean())
print(lbb.sum([-1]).mean())
# print(lbt.sum())
# print(lbb.sum())

In [ ]:
torch.cat([lu,lbb,lbt,lbr],2).sum([-1]).mean()

In [ ]:
# cc1 detail
E = 1.0
nu = 0.3
# C0np = np.array()
C0 = E/(1-nu**2)*torch.Tensor([[1,nu,0],[nu,1,0],[0,0,(1-nu)/2]]).to(device)
pp = input.contiguous().view(input.shape[0], -1, 1, 1).to(device)
# pp = input.permute(0,1,3,2).contiguous().view(input.shape[0], -1, 1, 1).to(device)
C = pp**3*C0

# duxdx = sobel_filter.grad_h(output[:, [0]])
# duxdy = sobel_filter.grad_v(output[:, [0]])
# duydx = sobel_filter.grad_h(output[:, [1]])
# duydy = sobel_filter.grad_v(output[:, [1]])
# d1 = duxdx
# d2 = duydy
# d3 = duxdy+duydx
# du = torch.cat([d1,d2,d3],1)
# du_post = du.view(du.shape[0],3,-1,1).permute(0,2,1,3)
B=np.zeros([4,3,8],dtype=np.float32)
B[0,:,:]=np.array([ [-1,  0, 1, 0, 0, 0, 0, 0],
                    [0, -1, 0, 0, 0, 0, 0, 1],
                    [-1, -1, 0, 1, 0, 0, 1, 0] ])
B[1,:,:]=np.array([ [-1,  0,  1,  0, 0, 0, 0, 0],
                    [0,  0,  0, -1, 0, 1, 0, 0],
                    [0, -1, -1,  1, 1, 0, 0, 0] ])
B[2,:,:]=np.array([ [0, 0,  0,  0, 1, 0, -1,  0],
                    [0, 0,  0, -1, 0, 1,  0,  0],
                    [0, 0, -1,  0, 1, 1,  0, -1] ])
B[3,:,:]=np.array([ [0,  0, 0, 0, 1, 0, -1,  0],
                    [0, -1, 0, 0, 0, 0,  0,  1],
                    [-1,  0, 0, 0, 0, 1,  1, -1] ])
B = torch.from_numpy(B).to(device)

B0 = torch.Tensor([[-0.5,0,-0.5],[0,-0.5,-0.5],[0.5,0,-0.5],\
    [0,-0.5,0.5],[0.5,0,0.5] ,[0,0.5,0.5],[-0.5,0,0.5],[0,-0.5,-0.5]]).to(device)
B0T = B0.transpose(0,1)
ux = output[:,[0]]
uy = output[:,[1]]

# k1 = torch.FloatTensor( np.array([[[[-0.5,0.5],[-0.5,0.5]]]]) ).to(device)
# k2 = torch.FloatTensor( np.array([[[[-0.5,-0.5],[-0.5,0.5]]]]) ).to(device)
# k31 = torch.FloatTensor( np.array([[[[-0.5,-0.5],[0.5,0.5]]]]) ).to(device)
# k32 = torch.FloatTensor( np.array([[[[-0.5,0.5],[-0.5,0.5]]]]) ).to(device)
# ux1 = F.conv2d(ux, k1, stride=1, padding=0, bias=None)
# uy1 = F.conv2d(uy, k2, stride=1, padding=0, bias=None)
# ux2 = F.conv2d(ux, k31, stride=1, padding=0, bias=None)
# uy2 = F.conv2d(uy, k32, stride=1, padding=0, bias=None)
# utest = torch.cat([ux1, uy1, ux2+uy2], 1)

unfold = nn.Unfold(kernel_size=(2, 2))
# [bs, 4, w*h]
uex = unfold(ux)
uey = unfold(uy)
ue_not = torch.cat([uex, uey], 1).permute([0,2,1])
ue = ue_not[:,:,[0,4,1,5,3,7,2,6]].unsqueeze(3)



# sig = output_post[:, [2,3,4]]
# sig_post = sig.view(sig.shape[0],3,-1,1).permute(0,2,1,3)
du0 = torch.matmul(C0@B[0,:,:],ue)
du1 = torch.matmul(C0@B[1,:,:],ue)
du2 = torch.matmul(C0@B[2,:,:],ue)
du3 = torch.matmul(C0@B[3,:,:],ue)
du0t = du0.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du1t = du1.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du2t = du2.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
du3t = du3.permute([0,2,1,3]).contiguous().view(-1,3,20,40)
ones = torch.ones_like(du0t)

masks =torch.zeros([du0.shape[0],3,21,41]).to(device)
result =torch.zeros([du0.shape[0],3,21,41]).to(device)

penal=3
Dp=torch.zeros([du0.shape[0],3,20,40]).to(device)
Dp[0,0,:,:]=input**penal
Dp[0,1,:,:]=input**penal
Dp[0,2,:,:]=input**penal

result[:,:,:20,:40] = du0t*Dp
result[:,:,:20,1:] = result[:,:,:20,1:]+du1t*Dp
result[:,:,1:,:40] = result[:,:,1:,:40]+du3t*Dp
result[:,:,1:,1:] = result[:,:,1:,1:]+du2t*Dp



masks[:,:,:20,:40] += ones
masks[:,:,:20,1:] += ones
masks[:,:,1:,:40] += ones
masks[:,:,1:,1:] += ones
lp1 = (result / masks) - output[:, [2,3,4]]

In [ ]:
print(C.shape)

In [ ]:
print(ue[0,0,:,0])
ue[0,1,:,0]

In [ ]:
print(du0t.shape)
du0t[0,0,0,0]

In [ ]:
print(lp1.shape)
print((lp1**2).sum([1,2,3]).mean())

In [ ]:
sxy[0,0,:,0]

In [ ]:
print(masks.shape)
masks[0,1,:,:]

In [22]:
# cc2 loss detail
dsxdx = sobel_filter.grad_h(output_post[:, [2]])
dsydy = sobel_filter.grad_v(output_post[:, [3]])
dsxydx = sobel_filter.grad_h(output_post[:, [4]])
dsxydy = sobel_filter.grad_v(output_post[:, [4]])

ds = torch.cat([dsxdx+dsxydy, dsydy+dsxydx],1)

In [23]:
ds.shape

torch.Size([1, 2, 20, 40])

In [24]:
print((ds ** 2).sum([1,2,3]).mean())

tensor(9.3706, device='cuda:0')


In [25]:
dsxdx.mean([0])

tensor([[[ 4.7309e-01, -9.8053e-02, -7.7725e-02, -5.3725e-02, -3.4291e-02,
          -2.0941e-02, -1.2106e-02, -6.2938e-03, -2.4844e-03,  1.4901e-08,
           1.6156e-03,  2.6500e-03,  3.2969e-03,  3.6719e-03,  3.8469e-03,
           3.8843e-03,  3.8281e-03,  3.6969e-03,  3.5125e-03,  3.3093e-03,
           3.0781e-03,  2.8157e-03,  2.5468e-03,  2.2875e-03,  2.0344e-03,
           1.7906e-03,  1.5719e-03,  1.3594e-03,  1.1531e-03,  9.8120e-04,
           8.3748e-04,  6.9065e-04,  5.5625e-04,  4.5626e-04,  3.7191e-04,
           2.9064e-04,  1.9682e-04,  1.0624e-04,  5.9374e-05, -3.7496e-01],
         [ 5.7779e-01, -3.6600e-02, -4.0275e-02, -4.2078e-02, -3.5528e-02,
          -2.6706e-02, -1.8759e-02, -1.2434e-02, -7.6781e-03, -4.1750e-03,
          -1.6031e-03,  2.5000e-04,  1.5594e-03,  2.4750e-03,  3.0875e-03,
           3.4500e-03,  3.6343e-03,  3.6844e-03,  3.6250e-03,  3.5094e-03,
           3.3344e-03,  3.0938e-03,  2.8281e-03,  2.5719e-03,  2.3188e-03,
           2.0531e-03,  